# analysis paricles in gyre 

In [ ]:
# import packages
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from datetime import datetime, timedelta
from importlib import reload

from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av
from analysis_functions import make_PDF, Haversine, make_lognormal_PDF
from particle_characteristics_functions import factor_drag_white1991, factor_drag_Schiller1933, factor_drag_morrison2013, Re_particle

# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water


In [ ]:
input_file = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NA_gyre/inertial_SM_drag_Rep/custom_start2023_09_01_end2023_10_01_RK4_B0680_tau2994_cor_True_gradient_True.zarr'


In [ ]:
ds = xr.open_dataset(input_file,engine='zarr')

In [ ]:
ds

In [ ]:
tid = 600
fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
ax.scatter(ds.lon[:,tid],ds.lat[:,tid],s=0.4)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}

ax.set_xlim(-100,0)
ax.set_ylim(0,60)

In [ ]:
pid =100
fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
ax.plot(ds.lon[pid,:],ds.lat[pid,:])
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}

ax.set_xlim(-100,0)
ax.set_ylim(0,60)

In [ ]:

fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
ax.plot(ds.lon[::,::24].T,ds.lat[::,::24].T)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}

# ax.set_xlim(-100,0)
# ax.set_ylim(0,60)

In [ ]:
Uslip = np.sqrt(ds.uslip**2+ds.vslip**2)
rep = Re_particle(Uslip,diameter,kinematic_viscosity_water)
fig,ax=plt.subplots()
array = rep.values.flatten()
array = array[array<5000]
print(np.mean(array))
bins, pdf= make_PDF(array,norm=True,vmin=0,vmax=1000,nbins=100)
ax.plot(bins,pdf)
ax.set_xlabel('Re$_p$')
ax.set_ylabel('PDF')

In [ ]:
factor_drag_white1991(np.mean(array))